Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [22]:
# Multi-layer with Mini-batch

n_hidden_nodes = 1024
batch_size = 128
l2_weight = 0.5e-3

graph = tf.Graph()
with graph.as_default():
    
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables
    weights_0 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_nodes]))
    weights_1 = tf.Variable(tf.truncated_normal([n_hidden_nodes, num_labels]))
    biases_0 = tf.Variable(tf.zeros([n_hidden_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_labels]))
        
    # Training computation
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weights_0) + biases_0)
    logits = tf.matmul(hidden_layer, weights_1) + biases_1
    train_prediction = tf.nn.softmax(logits)
    
    # Loss
    l2 = l2_weight * (tf.nn.l2_loss(weights_0) + tf.nn.l2_loss(weights_1))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Vailidation computation
    hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_0) + biases_0)
    valid_prediction = tf.nn.softmax(tf.matmul(hidden_layer, weights_1) + biases_1)
    
    # Test computation
    hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, weights_0) + biases_0)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_layer, weights_1) + biases_1)

In [16]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 540.335693
Minibatch accuracy: 3.9%
Validation accuracy: 18.9%
Minibatch loss at step 500: 133.205276
Minibatch accuracy: 85.2%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 98.792831
Minibatch accuracy: 80.5%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 73.930107
Minibatch accuracy: 89.8%
Validation accuracy: 81.9%
Minibatch loss at step 2000: 56.390335
Minibatch accuracy: 85.2%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 43.925182
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 33.946545
Minibatch accuracy: 85.9%
Validation accuracy: 84.7%
Test accuracy: 91.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Looping over the same 3 mini-batches:

In [35]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        loop_step = step - (3 *(step // 3))
        offset = (loop_step  * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 465.870117
Minibatch accuracy: 8.6%
Validation accuracy: 28.7%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Test accuracy: 81.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [38]:
# Multi-layer with Mini-batch

n_hidden_nodes = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables
    weights_0 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_nodes]))
    weights_1 = tf.Variable(tf.truncated_normal([n_hidden_nodes, num_labels]))
    biases_0 = tf.Variable(tf.zeros([n_hidden_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_labels]))
        
    # Training computation
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weights_0) + biases_0)
    logits = tf.nn.dropout(tf.matmul(hidden_layer, weights_1) + biases_1, 0.5)
    train_prediction = tf.nn.softmax(logits)
    
    # Loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Vailidation computation
    hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_0) + biases_0)
    valid_prediction = tf.nn.softmax(tf.matmul(hidden_layer, weights_1) + biases_1)
    
    # Test computation
    hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, weights_0) + biases_0)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_layer, weights_1) + biases_1)

Adding dropout on the limited data sets increases accuracy a little:

In [39]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        loop_step = step - (3 *(step // 3))
        offset = (loop_step  * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 467.791077
Minibatch accuracy: 9.4%
Validation accuracy: 14.2%
Minibatch loss at step 500: 0.856462
Minibatch accuracy: 57.8%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 0.886964
Minibatch accuracy: 57.8%
Validation accuracy: 75.2%
Minibatch loss at step 1500: 0.788409
Minibatch accuracy: 61.7%
Validation accuracy: 75.2%
Minibatch loss at step 2000: 0.837998
Minibatch accuracy: 55.5%
Validation accuracy: 75.2%
Minibatch loss at step 2500: 0.781438
Minibatch accuracy: 57.8%
Validation accuracy: 75.2%
Minibatch loss at step 3000: 0.912669
Minibatch accuracy: 52.3%
Validation accuracy: 75.2%
Test accuracy: 82.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


The design of the network below is from the TensorFlow MNIST tutorial. It uses two convolution/pooling layers followed by two fully connected layers, and uses `tf.train.AdamOptimizer` instead of `tf.train.GradientDescentOptimizer`.

It takes about 2.5 hours run 20,000 training steps on the CPU of a 2012 MacBook Pro.

In [64]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [80]:
n_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    # Variables
    
    dataset = tf.placeholder(tf.float32, shape=[None, 784], name='dataset')
    labels = tf.placeholder(tf.float32, shape=[None, 10], name='labels')
    
    conv1_weights = weight_variable([5, 5, 1, 32])
    conv1_biases = bias_variable([32])
    
    conv2_weights = weight_variable([5, 5, 32, 64])
    conv2_biases = bias_variable([64])
    
    full_con1_weights = weight_variable([7 * 7 * 64, n_hidden_nodes])
    full_con1_biases = bias_variable([n_hidden_nodes])
    
    full_con2_weights = weight_variable([n_hidden_nodes, 10])
    full_con2_biases = bias_variable([10])
    
    keep_prob = tf.placeholder(tf.float32)

    # First convolution layer
    reshaped_data = tf.reshape(dataset, [-1, 28, 28, 1])
    conv1 = tf.nn.relu(conv2d(reshaped_data, conv1_weights) + conv1_biases)
    pool1 = max_pool_2x2(conv1)
    
    # Second convolution layer
    conv2 = tf.nn.relu(conv2d(pool1, conv2_weights) + conv2_biases)
    pool2 = max_pool_2x2(conv2)
    
    # Dense layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    full_con1 = tf.nn.relu(tf.matmul(pool2_flat, full_con1_weights) + full_con1_biases)
    full_con1_dropout = tf.nn.dropout(full_con1, keep_prob)
    
    # Readout layer
    logits = tf.matmul(full_con1_dropout, full_con2_weights) + full_con2_biases
    predictions = tf.nn.softmax(logits)
    
    # Optimizer
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [109]:
num_steps = 20001
log_freq = 500

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step  * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        train_feed_dict = {dataset : batch_data, labels : batch_labels, keep_prob : 0.5}
        _, returned_loss, returned_predictions = session.run([optimizer, loss, predictions], 
                                                             feed_dict=train_feed_dict)
        if (step % log_freq == 0):
            print("Minibatch loss at step %d: %f" % (step, returned_loss))
            print("Minibatch accuracy: %.1f%%" % accuracy(returned_predictions, batch_labels))
            valid_feed_dict = {dataset : valid_dataset, labels : valid_labels, keep_prob : 1}
            valid_predictions, = session.run([predictions], feed_dict=valid_feed_dict)
            print("Validation accuracy: %.1f%%" % accuracy(valid_predictions, valid_labels))
    test_feed_dict = {dataset : test_dataset, labels : test_labels, keep_prob : 1}
    test_predictions, = session.run([predictions], feed_dict=test_feed_dict)
    print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))

Initialized
Minibatch loss at step 0: 8.808786
Minibatch accuracy: 12.5%
Validation accuracy: 10.5%
Minibatch loss at step 500: 8.808786
Minibatch accuracy: 84.4%
Validation accuracy: 83.3%
Minibatch loss at step 1000: 8.808786
Minibatch accuracy: 78.1%
Validation accuracy: 85.7%
Minibatch loss at step 1500: 8.808786
Minibatch accuracy: 88.3%
Validation accuracy: 86.9%
Minibatch loss at step 2000: 8.808786
Minibatch accuracy: 90.6%
Validation accuracy: 87.8%
Minibatch loss at step 2500: 8.808786
Minibatch accuracy: 86.7%
Validation accuracy: 88.2%
Minibatch loss at step 3000: 8.808786
Minibatch accuracy: 87.5%
Validation accuracy: 88.6%
Minibatch loss at step 3500: 8.808786
Minibatch accuracy: 86.7%
Validation accuracy: 89.2%
Minibatch loss at step 4000: 8.808786
Minibatch accuracy: 89.1%
Validation accuracy: 89.3%
Minibatch loss at step 4500: 8.808786
Minibatch accuracy: 90.6%
Validation accuracy: 89.6%
Minibatch loss at step 5000: 8.808786
Minibatch accuracy: 92.2%
Validation accurac